In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Example Mathieu

In [2]:
r_A = 2
r_B = 1
c   = 3
gamma=0.9

In [6]:
#Define a policy
p_a = 0.05
p_b = 0.05
P_pi = np.array([[p_a,1-p_a],[1-p_b,p_b]])
print P_pi , "\n"

r_pi = np.array([[r_A * P_pi[0,0] + (r_B - c) * P_pi[0,1]],
                 [(r_A - c) * P_pi[1,0] + r_B * P_pi[1,1]] ])
print r_pi

[[ 0.05  0.95]
 [ 0.95  0.05]] 

[[-1.8]
 [-0.9]]


I can compute analytically the state-value function matrix, but the inverse becomes too expensive for bigg state space(and, so, big matrices). This is why later we implement also policy evaluation algorithm

$$ v_{\pi} = r_{\pi} + \gamma P_{pi}v_{\pi} \implies v_{\pi} = (I - \gamma P_{\pi})^{-1} r_{\pi}$$ 

In [11]:
from numpy.linalg import inv
v_pi = np.dot(inv(np.eye(2) - gamma * P_pi), r_pi)

print v_pi

[[-13.74861878]
 [-13.25138122]]


### Policy evaluation

In [28]:
def policyEvaluation(epsilon):

    V_k = np.array([0,0])[:,None]
    
    delta = epsilon + 1
    while (delta >epsilon):        
        delta =0
        V_old  = V_k
        
        V_k    = r_pi + gamma * (P_pi.dot(V_k)) 
        
        delta = np.max(V_old - V_k)
      
        
    return V_k            

In [29]:
policyEvaluation(10**(-7))

array([[-13.7486179 ],
       [-13.25138033]])

## Example 4.1

In [3]:
import numpy as np

In [353]:
numStates = 16

#expected reward per ogni stato s
r_pi = np.ones(16) * -1
r_pi[0] = 0
r_pi[15]= 0

#pi: action probabilities for each state
pi = np.ones((16,4)) * 0.25
pi[0,:] = 0
pi[15,:]= 0

action_idx = {'u':0, 'd':1, 'r':2, 'l':3}

#Transitions
trans = np.zeros((16,4)).astype(int)

for s in range(1,15):        
    #prima riga
    if (s <=3):
        trans[s][0] = s
    else:
        trans[s][0] = s-4
    
    #ultima riga
    if (s>=12):
        trans[s][1] = s
    else:
        trans[s][1] = s+4
    
    #riga destra
    if ((s+1)%4 == 0):
        trans[s][2] = s
    else:
        trans[s][2] = s+1
    
    #riga sinistra
    if (s%4 == 0):
        trans[s][3] = s
    else:
        trans[s][3] = s-1

def transProb(trans, pi):
    transProbMatrix = np.zeros((16,16))
    for s in range(1,15):   
        for a in range(4):
            transProbMatrix[s,trans[s,a]] += pi[s,a]
    
    return transProbMatrix

In [339]:
V_k = np.zeros(16)
transProbMatrix = transProb(trans,pi)
V_k = r_pi + transProbMatrix.dot(V_k)

In [355]:
V_k = np.zeros(16)
delta = 10**(-7) + 1

while delta > 10**(-7):
    delta = 0
    V_k_old = V_k
    
    transProbMatrix = transProb(trans,pi)
    V_k = r_pi + transProbMatrix.dot(V_k)
    
    delta = np.amax(V_k_old - V_k)
    
V_k

array([  0.        , -13.99999897, -19.99999848, -21.9999983 ,
       -13.99999897, -17.99999866, -19.99999849, -19.99999848,
       -19.99999848, -19.99999849, -17.99999866, -13.99999897,
       -21.9999983 , -19.99999848, -13.99999897,   0.        ])

In [356]:
for s in range(1,15):
    act_res = r_pi[trans[s,:]] + V_k[trans[s,:]]
    greedy_actions = np.argwhere( act_res == np.amax(act_res)  ).squeeze()
    pi[s, :] =0
    pi[s, greedy_actions] = 1.0 / np.size(greedy_actions) 
    

** Below the pi is not exactly like in the results in the book, but this is for approximation errors. For example, V[1] = -19.99999848 and V[2]= -19.99999849 > V[1], but this is just an approximation error: they should be equal **

In [357]:
print pi

array([[ 0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  1. ],
       [ 0. ,  0. ,  0. ,  1. ],
       [ 0. ,  0.5,  0. ,  0.5],
       [ 1. ,  0. ,  0. ,  0. ],
       [ 1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  1. ],
       [ 0. ,  1. ,  0. ,  0. ],
       [ 1. ,  0. ,  0. ,  0. ],
       [ 1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  1. ,  0. ],
       [ 0. ,  1. ,  0. ,  0. ],
       [ 0. ,  0. ,  1. ,  0. ],
       [ 0. ,  0. ,  1. ,  0. ],
       [ 0. ,  0. ,  1. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ]])

### Bellman update with for loops 

In [ ]:
V_k0_copy = V_k0.copy()

for s in range(1,15):
    temp = 0
    for a in range(4):
        temp += pi[s][a] * (-1 + V_k0_copy[trans[s][a]])
    V_k0[s] = temp

V_k0
    